In [1]:
import pandas as pd
import numpy as np

# Load data from the csv file saved from Bigquery

In [2]:
taxi_weather_data_filtered = pd.read_csv(r'C:\Users\Pawan Lapborisuth\Dropbox\Data Science Final Project\features.csv',engine = 'python')

# Further filtering the data to include only NYC lat/lang

In [3]:
taxi_weather_data_filtered=taxi_weather_data_filtered[(taxi_weather_data_filtered['pickup_latitude']>40.5) & (taxi_weather_data_filtered['pickup_latitude']<41.5)]

# Use k-means to cluster lat/lang for linear models

In [4]:
from scipy.cluster.vq import kmeans2, whiten
import matplotlib.pyplot as plt
pickup_coordinates = taxi_weather_data_filtered[['pickup_latitude','pickup_longitude']]
dropoff_coordinates = taxi_weather_data_filtered[['dropoff_latitude','dropoff_longitude']]
pickup_x, pickup_y = kmeans2(whiten(pickup_coordinates), 50, iter = 20)  
dropoff_x, dropoff_y = kmeans2(whiten(dropoff_coordinates), 50, iter = 20)  
taxi_weather_data_filtered['pickup_cluster'] = pickup_y
taxi_weather_data_filtered['dropoff_cluster'] = dropoff_y
plt.scatter(pickup_coordinates.values[:,0], pickup_coordinates.values[:,1], c=pickup_y);
plt.show()

<Figure size 640x480 with 1 Axes>

In [5]:
taxi_weather_data_filtered.head()

,Unnamed: 0,pickup_datetime,pickup_latitude,pickup_longitude,dropoff_latitude,dropoff_longitude,travel_time,date_of_year,year,mo,...,day_binned,day_hour,time_binned,day_number,day_cosine,day_sine,time_cosine,time_sine,pickup_cluster,dropoff_cluster
0,0,2016-01-18 03:25:25,40.731525,-73.988670,40.760036,-73.984856,626,2016-01-18,2016,1,...,0.142857,3.0,0.125000,0.160714,0.532032,0.846724,0.707107,0.707107,8,34
1,1,2016-01-25 17:09:52,40.713608,-74.013718,40.765598,-73.980713,1192,2016-01-25,2016,1,...,0.142857,17.0,0.708333,0.244048,0.037391,0.999301,-0.258819,-0.965926,3,7
2,2,2016-01-05 00:17:41,40.773960,-73.874435,40.766693,-73.955414,842,2016-01-05,2016,1,...,0.285714,0.0,0.000000,0.285714,-0.222521,0.974928,1.000000,0.000000,43,25
3,3,2016-01-01 07:44:33,40.749718,-73.991570,40.768169,-73.912483,1054,2016-01-01,2016,1,...,0.714286,7.0,0.291667,0.755952,0.037391,-0.999301,-0.258819,0.965926,38,4
4,4,2016-05-29 14:34:21,40.762730,-73.974174,40.779640,-73.961823,538,2016-05-29,2016,5,...,0.000000,14.0,0.583333,0.083333,0.866025,0.500000,-0.866025,-0.500000,33,46


# Build linear models

### Linear regression

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

# Select the features to be included in the model
X = taxi_weather_data_filtered[['distance_in_km','pickup_cluster','dropoff_cluster','day_sine','day_cosine','time_sine','time_cosine','temp','prcp','sndp','wdsp','weekday']]
y = taxi_weather_data_filtered[['travel_time']]

# Split data into the training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Create linear regression object
regr = LinearRegression()

# Train the model using the training sets
regr.fit(X_train, y_train)

# Make predictions using the testing set
y_pred = regr.predict(X_test)

# Make predictions using the training set
y_pred_train = regr.predict(X_train)

# Compute R^2 scores for the training and testing sets
print('R^2 score for training data: %.4f' % r2_score(y_train, y_pred_train))
print('R^2 score for testing data: %.4f' % r2_score(y_test, y_pred))

R^2 score for training data: 0.5825
R^2 score for testing data: 0.5814


### Lasso 

In [9]:
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

# Select the features to be included in the model
X = taxi_weather_data_filtered.head(1000000)[['distance_in_km','pickup_cluster','dropoff_cluster','day_sine','day_cosine','time_sine','time_cosine','temp','prcp','sndp','wdsp','weekday']]
y = taxi_weather_data_filtered.head(1000000)[['travel_time']]

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
# Scale data
X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = y_train - np.mean(y_train) 
X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = y_test - np.mean(y_test)

# Use GridSearchCV to find alpha
alphas = np.linspace(0.0001,0.9,1000)
model = Lasso()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), cv=3)
grid.fit(X_train_scaled, y_train_scaled)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)

# Set up Lasso regression
lassoreg = Lasso(alpha=grid.best_estimator_.alpha,normalize=False, max_iter=1e5)

# Perform Lasso regression
lassoreg.fit(X_train_scaled,y_train_scaled)

# Make predictions using the training set
y_pred_train = lassoreg.predict(X_train_scaled)

# Make predictions using the testing set
y_pred = lassoreg.predict(X_test_scaled)

# Print R^2 scores
print('R^2 score for training data: %.4f' % r2_score(y_train_scaled, y_pred_train))
print('R^2 score for testing data: %.4f' % r2_score(y_test_scaled, y_pred))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  del sys.path[0]
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  from ipykernel import kernelapp as app


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([1.00000e-04, 1.00080e-03, ..., 8.99099e-01, 9.00000e-01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.5867717989235653
0.001000800800800801
R^2 score for training data: 0.5868
R^2 score for testing data: 0.5875


### Ridge

In [ ]:
from sklearn.linear_model import Ridge
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV

# Select the features to be included in the model
X = taxi_weather_data_filtered.head(1000000)[['distance_in_km','pickup_cluster','dropoff_cluster','day_sine','day_cosine','time_sine','time_cosine','temp','prcp','sndp','wdsp','weekday']]
y = taxi_weather_data_filtered.head(1000000)[['travel_time']]

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
    
# Scale data
X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = y_train - np.mean(y_train) 
X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = y_test - np.mean(y_test)

# Use GridSearchCV to find alpha
alphas = np.linspace(0.0001,0.9,1000)
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), cv=3)
grid.fit(X_train_scaled, y_train_scaled)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.alpha)


# Set up Lasso regression
ridgereg = Ridge(alpha=grid.best_estimator_.alpha,normalize=False, max_iter=1e5)

# Perform Lasso regression
ridgereg.fit(X_train_scaled,y_train_scaled)

# Make predictions using the training set
y_pred_train = ridgereg.predict(X_train_scaled)

# Make predictions using the testing set
y_pred = ridgereg.predict(X_test_scaled)

# Print R^2 scores
print('R^2 score for training data: %.4f' % r2_score(y_train_scaled, y_pred_train))
print('R^2 score for testing data: %.4f' % r2_score(y_test_scaled, y_pred))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  del sys.path[0]
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DataConversionWarning: Data with input dtype int32, float64 were all converted to float64 by the scale function.
  from ipykernel import kernelapp as app


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([1.00000e-04, 1.00080e-03, ..., 8.99099e-01, 9.00000e-01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.5867717989632822
0.9
R^2 score for training data: 0.5868
R^2 score for testing data: 0.5875


# Build K-nearest neighbors regressor model

In [8]:
from sklearn import neighbors
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

# Select the features to be included in the model
X = taxi_weather_data_filtered.head(500000)[['distance_in_km','pickup_latitude','pickup_longitude','dropoff_latitude','dropoff_longitude','day_sine','day_cosine','time_sine','time_cosine','temp','prcp','sndp','wdsp']]
y = taxi_weather_data_filtered.head(500000)[['travel_time']]

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Scale data
X_train_scaled = preprocessing.scale(X_train)
y_train_scaled = y_train - np.mean(y_train) 
X_test_scaled = preprocessing.scale(X_test)
y_test_scaled = y_test - np.mean(y_test)

# Use GridSearchCV to find the best k
k_range = [1,2,5,10,20,50,100]
model = neighbors.KNeighborsRegressor()
grid = GridSearchCV(estimator=model, param_grid=dict(n_neighbors=k_range), cv=3)
grid.fit(X_train_scaled, y_train_scaled)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.n_neighbors)

# Set up K nearest neighbor regressor
neigh = neighbors.KNeighborsRegressor(n_neighbors=grid.best_estimator_.n_neighbors)

# Perform K nearest neighbor regressor 
neigh.fit(X_train_scaled,y_train_scaled)
               
# Make predictions using the training set
y_pred_train = neigh.predict(X_train_scaled)

# Make predictions using the testing set
y_pred = neigh.predict(X_test_scaled)

# Print R^2 scores
print('R^2 score for training data: %.4f' % neigh.score(X_train_scaled, y_train_scaled))
print('R^2 score for testing data: %.4f' % neigh.score(X_test_scaled, y_test_scaled))


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [1, 2, 5, 10, 20, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
0.7362793499792443
10
R^2 score for training data: 0.7954
R^2 score for testing data: 0.7462


In [16]:
from sklearn.metrics import mean_squared_error
print('RMSE train : ',np.sqrt(mean_squared_error(preprocessing.scale(y_train_scaled),preprocessing.scale(y_pred_train))))
print('RMSE test : ',np.sqrt(mean_squared_error(preprocessing.scale(y_test_scaled),preprocessing.scale(y_pred))))

RMSE train :  0.46237454795709015
RMSE test :  0.5206814779941533


In [17]:
def rmsle(y_true, y_pred):
    return 'rmsle',np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))

In [19]:
print('RMSLE train : ',rmsle(y_train_scaled,y_pred_train))
print('RMSLE test : ',rmsle(y_test_scaled,y_pred))

RMSLE train :  ('rmsle', travel_time    0.978891
dtype: float64)
RMSLE test :  ('rmsle', travel_time    1.021035
dtype: float64)


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log1p
  
